In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from utils import save_response

# Llama 3 - 8B Instruct Model

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [3]:
messages = [
    {"role": "system", "content": f"""We have created a fitness app that helps users train for marathons. The app uses stories to help users train. The stories are generated based on user input/biometric data. 
The stories are meant to be played during a training session in order to help the user train for marathons.
You are a AI judge who rates and determines which output better fits the following criteria. The first criteria is the prompt, the second is the distance of the walk/run, 
the third is the average walking speed, the fourth is the average running speed, and the fifth is the training style (such as endurance run, walk, or anaerobic sprints). 
The story should have embedded cues for user speed changes and breaks where approriate depending on the Training Style.

Your job is to critique all of the different outputs and determine which one is the best. You will be given a prompt and the criteria listed above.
Rank all criteria seperately and then average the scores and give an overall score on a scale of 1-100. """},
    
    {"role": "user", "content": f"""Here is the first output which starts with the prompt and then the story. \n:
     
Prompt: A action packed adventure story based off of Indiana Jones, include outrunning deadly traps, a chase scene with a treasure hunter and a villain, jungles and ancient ruins.
Distance: 13.1 miles
Average Walking Speed: 2.5 mph
Average Running Speed: 7.2 mph
Training Style: Half Marathon

It’S JULIET! It starts like everyone knows it SHOOTING OFF IN THE GREAT DEADLY TRAPS OF IRON-CLUB WORLD!!!!! He takes his bow but when one appears he droppt down behind some crates ! Then all these things appear around him there s rocks fallng from walls so fast bah BANG !! BOOM THEN HE SWOOPES UP FROM BELOW U SEE PANNED NOW AND IT IS DARKER FINALLY SOUTH FORWARD RIGHT TO GET MOVIE ONSET... (PLAY CATCHUP)....THEN RUNRUN THIS ALL MAKEFULL COMMENT REGIONAL START MUSIC????? HIGHPLEXEL? TRAP ROCK CRASH CLIPPITZCRIIIIIMPLOPE ....GO ALRIGHT .. ANNO YOU OUTLAST EVERDED LANDMANS LIKE RAIDMAN NOODLERS ? Noodler ?? OK , its gonna go nw then run fwd again . ok now we got shots at different angles ... okay here they enter this old village just after warming up maybe get lost or something cause right away people attack them both not slowy as u can see lol I smell gold though why dont thoooosers pick us :p Ok guys running ahead let em pass quickly wtf do i look dumb rn lets continue yay short distance half mile about seven minutes walking time five seconds crossbow shot..walk faster etc walk more slowly turn left going into battle area next stop cutdown wait what oh man first blood very exciting donotlooki know if ill kill anyone im too cool idk how many noobs killed before hauntingman dies looking stupid oops also lots other guns used well anyhow keep shooting die already shoota lot nahh never mind another minute later almost ran someone say "came" alright need less background noise take three steps back move forward kick near hand guards slit neak hands knives use gun throw sword slash stab jump climb grab hold claws punches body hit grenades blow torch blaster laser eyes eye cover roar loud boom scream sound stuff coming closer make our heart rate rise because close fighting happens mostly really intense please listen carefully especially big spikes getting smaller while still being huge awesome tails whack head chopping arms swing throwing weapons rolling objects flying overcoming obstacles vicious actions words shoutings speech sounds monkeys speaking animals saying important lines exclamations such statements describing fight scenes including descriptions visualizing surroundling elements characters posotion expression gestures movement scenic designs landscape locations landmarks features environmental settings audio video animation graphical images creatures spirits zombies skeletor evil minters menace darkness flames fire wind smoke rain snow night day sun light shadow lights stars constellaints air clouds mist sandstone rock crystal sky water sprinklers iris earth moon twilight mountains trees branches leaves thick roots flowers flower petals grass shooting star fog silhouette reddish purple blue orange green yellow magenta plasma silver black electric white turquise translucent violets scarlato gray cyan grey teal lavender burgundywine colorful pastel colors aquamarines dark saffron indigo rose tan brown egluksred metallics diamond shapes multicolored spears bulky textured skinny muscular bodies fat thin curvy limbs sharp teeth cladding hair mouthpieces horn antagonist appearance contrast perspective viewpoint narration directors commentary score suspense excitement humor thrilling plot development dramatic music playlist character dialogue setting ambient ambience environment weather atmosphere emotions psychology mental focus inner thoughts feelings dream sequences ideas imagination concept imaginary concepts surreal realistic events fictional scenarios random occurrences memories flashback remembran below showcase replay pause zoom follow directions key tips shortcut footage montages sublimity archery graveyard tomb mausoleum skull face sweat arena steeples cave passageways forest woodlands fields vegetable garden house car park kitchen cupboard roof railing sidewall gatefront yard building ground street corner city roads mountain valley range plaza parkinglot square field court benchtables table chair desklife treehouse room floor hall closet ceileroom basement tunnel ventilation fan engine coils machine assembly section pipelines switchbox mainframe device system interface panel display screen monitor electronic circuit board terminal electron flow current voltage pulsing photon power quantum dynamics atomic scale structures space dimensions frequency cycles periodic patterns energy motion physical laws natural phenomena mathematical theories logic reasoning causality consequence effect theory principle operation component structure mechanism function activity process reaction pattern procedure element sequence step cycle repetition
"""},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [4]:
hyperparams = {
    "max_new_tokens": 256, 
    "temperature":0.6, 
    "top_p":0.9, 
    "do_sample": True
}
outputs = model.generate(
    input_ids,
    max_new_tokens=hyperparams["max_new_tokens"],
    eos_token_id=terminators,
    do_sample=hyperparams["do_sample"],
    temperature=hyperparams["temperature"],
    top_p=hyperparams["top_p"],
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\pmahi\OneDrive\Desktop\MP\Drexel\Courses\CS614-900 - Applications in Machine Learning\Projects\Natural_Language_Assignment\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
response = outputs[0][input_ids.shape[-1]:]
output = tokenizer.decode(response, skip_special_tokens=True)
print(output)

I will critique the output based on the given criteria.

**Prompt:** The story starts with the prompt, but it quickly diverges from the adventure story based on Indiana Jones, and instead becomes a chaotic mix of action scenes, trap escapes, and fight sequences. The story fails to incorporate the treasure hunter and the villain, and the jungle and ancient ruins settings are not fully utilized. Score: 20/50

**Distance:** The story is approximately 13.1 miles long, which matches the given distance. Score: 50/50

**Average Walking Speed:** The story includes walking scenes, but it does not provide clear cues for walking speed changes. The average walking speed is not consistently maintained, and the story jumps between walking and running without clear transitions. Score: 30/50

**Average Running Speed:** The story includes running scenes, and the average running speed is occasionally mentioned (e.g., "RUNRUN THIS ALL MAKEFULL COMMENT REGIONAL START MUSIC?????"). However, the running spe

In [6]:
save_response("LLAMA3_JUDGE", hyperparams, output)

In [7]:
# perplexity = evaluate.load("perplexity", module_type="metric")
# input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
# results = perplexity.compute(model_id=model_id,
#                              add_start_token=False,
#                              predictions=input_texts, 
#                              device="cuda")
# print(list(results.keys()))

In [8]:
# print(round(results["mean_perplexity"], 2))